# So easy, *voilà* IPyMaps!

In this example notebook, we demonstrate how voila can render custom interactive maps with built-in search. Try entering something like the following into the search field:

- 52.5 13.4
- Paris
- https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json
- berlin_stops (a predefined GeoJSON object)

In [5]:
import re
import os

import requests
import geojson
from ipywidgets import Text, HTML, Layout
from ipyleaflet import Map, GeoJSON, WidgetControl, FullScreenControl, LayersControl
from geopy.geocoders import Nominatim

In [6]:
def bbox(coord_list):
    "Find bounding box (lower-left and upper-right points) for a list of coordinates."
    box = []
    for i in (0, 1):
        res = sorted(coord_list, key=lambda x:x[i])
        box.append((res[0][i], res[-1][i]))
    ll, ur = [box[0][0], box[1][0]], [box[0][1], box[1][1]]
    return [ll, ur]

In [7]:
class SearchMap(Map):
    """
    This widget provides a simple maps-search UI.
    
    For now this is a simple wrapper around ipyleaflet.Map mainly adding a search
    text field. Depending on the sumitted text one of the ollowing things can happen
    (attempted in the given order):
    
    0. reference: load and display respective object from given namespace if it's a GeoJSON one 
    1. lat/lon: center the map to the respective coordinates
    2. URL ending in '.geojson' or 'geo.json': load and display GeoJSON data
    3. text: geolocate address and center the map to the respective coordinates
    """
    def __init__(self, *args, geocoder=None, globs=None, **kwargs):
        """Instantiate widget.
        
        All parameters are like those for ``ipyleaflet.Map`` plus the following ones:
        
        - geocoder: a ``geopy.geocoder`` instance,
                    default: ``geopy.geocoders.Nominatim(user_agent="ipymaps")``
        - globs: a dictionary (namespace) from which references to GeoJSON objects
                    will be looked up
        """
        self.geocoder = geocoder or Nominatim(user_agent="ipymaps")
        self.globs = globs or {}

        super().__init__(*args, zoom_control=False, **kwargs)
        # FIXME: add zoom control again once it can be put below the search field

        self.status_control = None

        # add search text field
        self.search_tx = Text(layout=Layout(width='200px'))
        self.search_tx.on_submit(self.text_submitted)
        wc = WidgetControl(widget=self.search_tx, position='topleft')
        self.add_control(wc)
        
        # add other controls
        self.add_control(FullScreenControl())
        self.add_control(LayersControl())

    def set_status(self, text):
        self.status_tx = HTML(text, layout=Layout(width='50%'))
        self.status_control = WidgetControl(widget=self.status_tx, position='bottomleft')
        self.add_control(self.status_control)

    def reset_status(self):
        if self.status_control:
            self.remove_control(self.status_control)
            self.status_control = None

    def text_submitted(self, widget):
        self.run_query(widget.value)

    def add_geojson(self, url):
        # load data
        self.set_status('Loading...')
        if url.startswith('file://'):
            gj = open(url[7:]).read()
        else:
            gj = requests.get(url).content
        data = geojson.loads(gj)
        self.reset_status()
        
        # jump to center of data
        ll, ur = bbox(list(geojson.utils.coords(data)))
        center = [(ur[0]+ll[0])/2, (ur[1]+ll[1])/2]
        self.center = list(reversed(center))
        name = os.path.basename(url)
        self.add_layer(GeoJSON(data=data, name=name))

    def run_query(self, text):
        self.reset_status()

        text = text.strip()

        # geojson objects in passed namespace        
        if text in self.globs:
            obj = self.globs[text]
            if hasattr(obj, '__geo_interface__'):
                self.add_layer(GeoJSON(data=obj, name=text))
                return

        # lat lon
        try:
            values = re.split('[/,;: ]', text)
            lat, lon = [v for v in values if v not in '/,;:']
            self.center = lat, lon
            return
        except ValueError:
            pass

        # geojson
        url = text.strip()
        if url.endswith('.geojson') or url.endswith('.geo.json'):
            self.add_geojson(url)
            return

        # simple geolocation
        loc = self.geocoder.geocode(text)
        if loc:
            self.center = loc.latitude, loc.longitude
        else:
            self.set_status('Not found') 

In [14]:
url = 'https://gist.githubusercontent.com/deeplook/40ae4f4ecd8ec8d4be51/raw/c1dcdf253e47faf8586cffa4e009a705c8c0906b/stops_berlin.geojson'
berlin_stops = geojson.loads(requests.get(url).content)

In [15]:
m = SearchMap(center=[52, 13], zoom=4, globs=globals())
m

SearchMap(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution':…